In [1]:
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LeakyReLU
from tensorflow.keras.layers import Conv2D, BatchNormalization, LocallyConnected2D, SeparableConv2D
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.constraints import max_norm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import activations
from create_data_victory import History, create_all_x_y

# Data creation

In [2]:
x, y = create_all_x_y()
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

248 parties rejetées, reste 41563.0
x shape: (332504, 9, 9, 11)
y shape: (332504,)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(299253, 9, 9, 11)
(299253,)
(33251, 9, 9, 11)
(33251,)


# Model

In [4]:
model = Sequential()
for _ in range(6):
    model.add(Conv2D(64, kernel_size=(3,3), input_shape=x[0].shape, padding ="same", data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_absolute_error', optimizer=optimizers.SGD(learning_rate=0.01, momentum=0.9))
model.summary()


def tuned_model(): #Tuned with Keras Tuner
    model = Sequential()
    model.add(Conv2D(32, kernel_size=5, activation='relu', input_shape=(9,9,11), padding ="same"))
    model.add(Conv2D(256, kernel_size=5, activation='relu', padding ="same"))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid')) 
    opt= optimizers.Adam()
    model.compile(loss='mean_absolute_error',
                optimizer = opt)
    model.summary()
    return model
model = tuned_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 64)          6400      
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
activation (Activation)      (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 128)         512       
_________________________________________________________________
activation_1 (Activation)    (None, 9, 9, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 10368)             0

# Training

In [ ]:
history=[History()]
# hyperparamètres
epochs = 50
batch_size = 500
# training
print("Training ...")
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          callbacks=history)

Training ...
Epoch 1/50
257/599 [===========>..................] - ETA: 2:21 - loss: 0.2307

# Statistics

In [ ]:
#Print model loss
plt.plot(history[0].history['loss'])
plt.plot(history[0].history['val_loss'])
plt.title('Model Loss (on training data)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Loss', 'Loss (val)'])
plt.show()

In [ ]:
sample_size = 14
print(y_test[:sample_size])
print(np.round(model.predict(X_test[:sample_size]).reshape((sample_size,)), 2))